<a href="https://colab.research.google.com/github/kjaqdenusi/AI/blob/main/AI_HW1_WaterjugPuzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Water Jug Puzzle

This notebook shows a very basic implementation of the classic 5L–3L water jug puzzle.

In [1]:
# ****************************************************
# *** Full name: Kenny Adenusi
# *** Course title: introduction to AI / Problem Solving
# *** Submission date: 2025-09-12
# *** assignment number and purpose:
# *** HW1 – implement a simple state space + rules for
# ***    5L / 3L Water Jug puzzle and demo a solution.
# ****************************************************


from dataclasses import dataclass
from typing import Tuple, List

State = Tuple[int, int]  # (amount_in_5L, amount_in_3L)

@dataclass
class RuleResult:
    ok: bool
    new_state: State
    reason: str  # text to explain what happened


class WaterJugPuzzle:
    def __init__(self):
        # capacities
        self.CAP5 = 5
        self.CAP3 = 3

        # start and goal
        self.start: State = (0, 0)
        self.goal:  State = (4, 0)  # "exactly four litres into five litre jug"

    # -------------- helper checks --------------
    def _valid(self, s: State) -> bool:
        """make sure state respects jug capacities and non-negative water."""
        a, b = s
        return (0 <= a <= self.CAP5) and (0 <= b <= self.CAP3)

    # -------------- rule definitions --------------
    # rach rule returns a RuleResult letting us know if it changed anything.

    def fill5(self, s: State) -> RuleResult:
        """fill 5L jug to top."""
        a, b = s
        if a == self.CAP5:
            return RuleResult(False, s, "5L already full")
        ns = (self.CAP5, b)
        return RuleResult(True, ns, "filled 5L jug")

    def fill3(self, s: State) -> RuleResult:
        """fill 3L jug to top."""
        a, b = s
        if b == self.CAP3:
            return RuleResult(False, s, "3L already full")
        ns = (a, self.CAP3)
        return RuleResult(True, ns, "filled 3L jug")

    def empty5(self, s: State) -> RuleResult:
        """empty 5L jug."""
        a, b = s
        if a == 0:
            return RuleResult(False, s, "5L already empty")
        ns = (0, b)
        return RuleResult(True, ns, "emptied 5L jug")

    def empty3(self, s: State) -> RuleResult:
        """empty 3L jug."""
        a, b = s
        if b == 0:
            return RuleResult(False, s, "3L already empty")
        ns = (a, 0)
        return RuleResult(True, ns, "emptied 3L jug")

    def pour5to3(self, s: State) -> RuleResult:
        """pour from 5L into 3L until 3L is full or 5L is empty."""
        a, b = s
        if a == 0 or b == self.CAP3:
            return RuleResult(False, s, "nothing to pour from 5L to 3L")
        space = self.CAP3 - b
        move = min(a, space)
        ns = (a - move, b + move)
        return RuleResult(True, ns, f"poured {move}L from 5L to 3L")

    def pour3to5(self, s: State) -> RuleResult:
        """pour from 3L into 5L until 5L is full or 3L is empty."""
        a, b = s
        if b == 0 or a == self.CAP5:
            return RuleResult(False, s, "nothing to pour from 3L to 5L")
        space = self.CAP5 - a
        move = min(b, space)
        ns = (a + move, b - move)
        return RuleResult(True, ns, f"poured {move}L from 3L to 5L")

    # -------------- order of rule applications --------------
    def demo_solution(self) -> List[RuleResult]:
        """
        this is "main" for assignment.
        it runs a **fixed** sequence of legal rule applications:
        goal: is to get 4L in 5L jug.
        known simple sequence (matches picture steps idea):
          1) fill5
          2) pour5to3
          3) empty3
          4) pour5to3
          5) fill5
          6) pour5to3   -> should land on (4, 3)
          7) empty3     -> (4, 0) which is th goal
        """
        s = self.start
        steps: List[RuleResult] = []
        def apply(rule_fn):
            nonlocal s, steps
            r = rule_fn(s)
            if not r.ok:
                # gaurd against illegal states.
                steps.append(r)
                return
            if not self._valid(r.new_state):
                steps.append(RuleResult(False, s, "invalid state produced"))
                return
            s = r.new_state
            steps.append(r)

        apply(self.fill5)     # 1
        apply(self.pour5to3)  # 2
        apply(self.empty3)    # 3
        apply(self.pour5to3)  # 4
        apply(self.fill5)     # 5
        apply(self.pour5to3)  # 6
        apply(self.empty3)    # 7 (shows end)

        # store final state so you can see it ended at goal.
        self.end_state = steps[-1].new_state if steps else s
        return steps


# ---------- run demo when cell executes ----------
puzzle = WaterJugPuzzle()
results = puzzle.demo_solution()

print("start state:", puzzle.start)
for i, r in enumerate(results, 1):
    print(f"step {i}: {r.reason} -> state {r.new_state}")
print("goal state expected:", puzzle.goal)
print("end state reached:", getattr(puzzle, "end_state", None))




start state: (0, 0)
step 1: filled 5L jug -> state (5, 0)
step 2: poured 3L from 5L to 3L -> state (2, 3)
step 3: emptied 3L jug -> state (2, 0)
step 4: poured 2L from 5L to 3L -> state (0, 2)
step 5: filled 5L jug -> state (5, 2)
step 6: poured 1L from 5L to 3L -> state (4, 3)
step 7: emptied 3L jug -> state (4, 0)
goal state expected: (4, 0)
end state reached: (4, 0)
